<a href="https://colab.research.google.com/github/misanchz98/bitcoin-direction-prediction/blob/main/03_modeling/03_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Modelización con Redes Neuronales

In [105]:
# =============================================================================
# LIBRERIAS
# =============================================================================

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import metrics
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
import random

import warnings

# Establecer el nivel de advertencias a "ignore" para ignorar todas las advertencias
warnings.filterwarnings("ignore")

In [106]:
# resetting the seeds for reproducibility
def reset_random_seeds():
    n = 1
    os.environ['PYTHONHASHSEED'] = str(n)
    tf.random.set_seed(n)
    np.random.seed(n)
    random.seed(n)

reset_random_seeds()

## 🔹 1. Importación del conjunto de datos
El primer paso consiste en cargar el conjunto de datos en nuestro entorno de trabajo. Estos datos están almacenados en un archivo CSV llamado `btc_historical_data_eda.csv`, cuya creación y obtención se explican en el notebook `02_data_analysis.ipynb`. Para ello, se utiliza el siguiente fragmento de código:

In [107]:
# Importamos CSV
url = 'https://raw.githubusercontent.com/misanchz98/bitcoin-direction-prediction/main/02_data_analysis/data/btc_historical_data_eda.csv'
df_bitcoin = pd.read_csv(url, parse_dates=['Open time'])
df_bitcoin

,Open time,Close,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Range,Candle,Target,CMF_20,MFI_14,...,c2_ta_tendencia,c3_ta_tendencia,c1_ta_momentum,c2_ta_momentum,c3_ta_momentum,c4_ta_momentum,c5_ta_momentum,c1_ta_volatilidad,c2_ta_volatilidad,c3_ta_volatilidad
0,2017-10-05,4292.43,9158.0,351.042019,1.483037e+06,245.00,83.84,1,0.081329,56.225018,...,1.426309,-0.136861,1.317719,-0.126726,-1.676720,1.289633,0.019128,-4.644643,-0.938175,-0.259897
1,2017-10-06,4369.00,6546.0,226.148177,9.881066e+05,125.00,50.01,1,0.090972,62.048701,...,1.684975,-0.223654,1.843789,-0.313902,-0.766032,1.200655,0.108543,-4.656021,-1.404483,-0.298095
2,2017-10-07,4423.00,4804.0,145.313076,6.371469e+05,166.94,54.00,1,0.072898,60.780168,...,1.837639,-0.272101,1.714315,0.851111,-1.474281,-0.169404,-0.611932,-4.658149,-1.694457,-0.310605
3,2017-10-08,4640.00,7580.0,280.094854,1.268661e+06,233.00,215.00,1,0.064115,66.225272,...,2.655718,-0.595153,4.539268,-1.327677,-1.397994,0.344940,0.095779,-4.643444,-2.930102,-0.299058
4,2017-10-09,4786.95,10372.0,350.756559,1.654275e+06,339.98,146.95,0,0.105281,66.423592,...,3.068594,-0.711866,4.065484,0.284535,-0.496135,-0.373562,-0.383308,-4.619178,-3.116037,-0.259659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2872,2025-08-16,117380.66,1179842.0,2995.228650,3.521588e+08,755.01,38.62,1,-0.078079,61.679789,...,-2.974279,-0.597347,-1.738575,-0.246759,0.045690,1.947962,-0.826190,11.130802,0.715864,-3.617415
2873,2025-08-17,117405.01,1177563.0,2804.731130,3.307994e+08,1402.79,24.35,0,-0.071478,61.441782,...,-3.056170,-0.520901,-2.679964,0.452454,-1.078307,1.833305,-0.382838,11.062656,0.708259,-3.717682
2874,2025-08-18,116227.05,3345487.0,7647.218200,8.850528e+08,2903.61,-1177.96,0,-0.058026,54.527915,...,-3.371373,-0.335897,-3.708156,1.216496,-0.767095,1.913662,1.633951,11.085307,1.427748,-3.651715
2875,2025-08-19,112872.94,3291170.0,8609.360780,9.840874e+08,3993.11,-3354.11,1,-0.133646,53.037041,...,-4.158761,0.112011,-5.328226,0.713262,0.105456,1.232647,-1.954862,11.229262,3.161925,-3.269720


## 🔹 2. División del conjunto de datos

In [108]:
X = df_bitcoin.drop(['Open time', 'Target'], axis=1)
y = df_bitcoin['Target']

# separate training data from testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# scale the input data
scaler = StandardScaler()

# Reshape X_train and X_test if they are 1D
if X_train.ndim == 1:
    X_train = X_train.to_numpy().reshape(-1, 1)
if X_test.ndim == 1:
    X_test = X_test.to_numpy().reshape(-1, 1)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# reshape the input data for CNN-LSTM (samples, timesteps, features)
def create_sequences(data, timesteps):
    X = []
    for i in range(len(data) - timesteps + 1):
        X.append(data[i:i + timesteps])
    return np.array(X)

timesteps = 14
X_train_reshaped = create_sequences(X_train_scaled, timesteps)
X_test_reshaped = create_sequences(X_test_scaled, timesteps)
y_train = y_train[timesteps - 1:]
y_test = y_test[timesteps - 1:]

### LSTM

In [109]:
from tensorflow.keras import backend as K

def f1_score_2(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [110]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import metrics

model = Sequential()
model.add(LSTM(256, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # binaria

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=[metrics.BinaryAccuracy(), metrics.Precision(), metrics.Recall(), f1_score_2]
)

early_stopping = EarlyStopping(
    monitor='val_binary_accuracy',  # mejor usar val_binary_accuracy
    patience=50,
    mode='max',
    restore_best_weights=True
)

history = model.fit(
    X_train_reshaped, y_train.astype('float32'),  # convertir a float32
    epochs=100,
    batch_size=50,
    validation_split=0.1,
    callbacks=[early_stopping]
)


Epoch 1/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - binary_accuracy: 0.5096 - f1_score_2: 0.5394 - loss: 0.6999 - precision_7: 0.5254 - recall_7: 0.5743 - val_binary_accuracy: 0.5371 - val_f1_score_2: 0.6479 - val_loss: 0.6908 - val_precision_7: 0.5260 - val_recall_7: 0.8707
Epoch 2/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - binary_accuracy: 0.4901 - f1_score_2: 0.5586 - loss: 0.6992 - precision_7: 0.5072 - recall_7: 0.6355 - val_binary_accuracy: 0.5328 - val_f1_score_2: 0.5544 - val_loss: 0.6933 - val_precision_7: 0.5298 - val_recall_7: 0.6897
Epoch 3/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - binary_accuracy: 0.5085 - f1_score_2: 0.5647 - loss: 0.6928 - precision_7: 0.5217 - recall_7: 0.6372 - val_binary_accuracy: 0.5328 - val_f1_score_2: 0.6518 - val_loss: 0.6952 - val_precision_7: 0.5226 - val_recall_7: 0.8966
Epoch 4/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - binary_accuracy: 0.5099 - f1_score_2: 0.5592 - loss: 0.6930 - precision_7: 0.5236 - recall_7: 0.6242 - val_binar

In [111]:
# Make predictions on the test set
y_pred = model.predict(X_test_reshaped)
y_pred = (y_pred > 0.5)

# evaluate the prediction performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))

18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
Accuracy: 0.5257548845470693
Precision: 0.5755813953488372
Recall: 0.3378839590443686
F1-score: 0.4258064516129032


In [112]:
# =========================================
# Walk-Forward en df_bitcoin
# =========================================
#import numpy as np
#import pandas as pd
#import tensorflow as tf
#from tensorflow.keras import Sequential
#from tensorflow.keras.layers import LSTM, GRU, Dense, Conv1D, MaxPooling1D, Dropout
#from sklearn.preprocessing import StandardScaler, MinMaxScaler
#from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
#from sklearn.utils.class_weight import compute_class_weight
#from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
#
## -----------------------------
## Funciones de ventanas (las tuyas)
## -----------------------------
#def create_windows_multivariate_np(data, window_size, horizon, target_col_idx, shuffle=False):
#    if isinstance(data, pd.DataFrame):
#        data = data.values
#    X, y = [], []
#    for i in range(len(data) - window_size - horizon + 1):
#        X.append(data[i:i+window_size, :])
#        y.append(data[i+window_size+horizon-1, target_col_idx])
#    X, y = np.array(X), np.array(y)
#    if shuffle:
#        indices = np.arange(X.shape[0])
#        np.random.shuffle(indices)
#        X, y = X[indices], y[indices]
#    return X, y
#
## -----------------------------
## Builders de modelos
## -----------------------------
#def build_lstm(input_shape):
#    model = Sequential([
#        LSTM(64, input_shape=input_shape),
#        Dense(32, activation='relu'),
#        Dropout(0.2),
#        Dense(1, activation='sigmoid')
#    ])
#    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#    return model
#
#def build_gru(input_shape):
#    model = Sequential([
#        GRU(64, input_shape=input_shape),
#        Dense(32, activation='relu'),
#        Dropout(0.2),
#        Dense(1, activation='sigmoid')
#    ])
#    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#    return model
#
#def build_cnn_lstm(input_shape):
#    model = Sequential([
#        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
#        MaxPooling1D(pool_size=2),
#        LSTM(64),
#        Dense(32, activation='relu'),
#        Dropout(0.2),
#        Dense(1, activation='sigmoid')
#    ])
#    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#    return model
#
#def build_cnn_gru(input_shape):
#    model = Sequential([
#        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
#        MaxPooling1D(pool_size=2),
#        GRU(64),
#        Dense(32, activation='relu'),
#        Dropout(0.2),
#        Dense(1, activation='sigmoid')
#    ])
#    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#    return model
#
#MODEL_BUILDERS = {
#    "LSTM": build_lstm,
#    "GRU": build_gru,
#    "CNN+LSTM": build_cnn_lstm,
#    "CNN+GRU": build_cnn_gru
#}
#
## 1. BUILDERS DE MODELOS MEJORADOS
#def build_lstm_improved(input_shape):
#    """LSTM mejorado con mejor arquitectura"""
#    model = Sequential([
#        LSTM(128, return_sequences=True, input_shape=input_shape, dropout=0.2, recurrent_dropout=0.1),
#        LSTM(64, dropout=0.2, recurrent_dropout=0.1),
#        Dense(64, activation='relu'),
#        Dropout(0.3),
#        Dense(32, activation='relu'),
#        Dropout(0.2),
#        Dense(1, activation='sigmoid')
#    ])
#    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005, clipnorm=1.0)
#    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
#    return model
#
#def build_gru_improved(input_shape):
#    """GRU mejorado con regularización"""
#    model = Sequential([
#        GRU(128, return_sequences=True, input_shape=input_shape, dropout=0.2, recurrent_dropout=0.1),
#        GRU(64, dropout=0.2, recurrent_dropout=0.1),
#        Dense(64, activation='relu'),
#        Dropout(0.3),
#        Dense(32, activation='relu'),
#        Dropout(0.2),
#        Dense(1, activation='sigmoid')
#    ])
#    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005, clipnorm=1.0)
#    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
#    return model
#
#def build_cnn_lstm_improved(input_shape):
#    """CNN+LSTM mejorado con múltiples escalas"""
#    model = Sequential([
#        Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=input_shape),
#        Conv1D(filters=64, kernel_size=3, activation='relu'),
#        MaxPooling1D(pool_size=2),
#        Dropout(0.2),
#        LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.1),
#        LSTM(64, dropout=0.2, recurrent_dropout=0.1),
#        Dense(64, activation='relu'),
#        Dropout(0.3),
#        Dense(32, activation='relu'),
#        Dropout(0.2),
#        Dense(1, activation='sigmoid')
#    ])
#    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005, clipnorm=1.0)
#    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
#    return model
#
#def build_cnn_gru_improved(input_shape):
#    """CNN+GRU mejorado"""
#    model = Sequential([
#        Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=input_shape),
#        Conv1D(filters=64, kernel_size=3, activation='relu'),
#        MaxPooling1D(pool_size=2),
#        Dropout(0.2),
#        GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.1),
#        GRU(64, dropout=0.2, recurrent_dropout=0.1),
#        Dense(64, activation='relu'),
#        Dropout(0.3),
#        Dense(32, activation='relu'),
#        Dropout(0.2),
#        Dense(1, activation='sigmoid')
#    ])
#    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005, clipnorm=1.0)
#    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
#    return model
#
## Diccionario de modelos
#MODEL_BUILDERS_IMPROVED = {
#    "LSTM": build_lstm_improved,
#    "GRU": build_gru_improved,
#    "CNN+LSTM": build_cnn_lstm_improved,
#    "CNN+GRU": build_cnn_gru_improved
#}
#
#
## -----------------------------
## Utilidades
## -----------------------------
#def make_class_weights(y_train):
#    classes = np.array([0, 1])
#    weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
#    return {int(c): float(w) for c, w in zip(classes, weights)}
#
## -----------------------------
## Walk-Forward Validation
## -----------------------------
#def walk_forward_validation(
#    X_values, y_values,
#    window_size=30, horizon=1,
#    train_size=720, test_size=30, step_size=30,
#    build_model_fn=build_lstm,
#    epochs=10, batch_size=32, threshold=0.5,
#    use_class_weights=True, verbose=0
#):
#    n = len(X_values)
#    n_feat = X_values.shape[1]
#
#    y_true_all, y_proba_all, y_pred_all, fold_id_all = [], [], [], []
#    folds_info = []
#
#    start = 0
#    fold = 0
#    while start + train_size + test_size <= n:
#        fold += 1
#        end_train = start + train_size
#        end_test = end_train + test_size
#
#        segment_X = X_values[start:end_test]
#        segment_y = y_values[start:end_test]
#
#        segment_combined = np.hstack([segment_X, segment_y.reshape(-1, 1)])
#        Xw_all, yw_all = create_windows_multivariate_np(
#            data=segment_combined,
#            window_size=window_size,
#            horizon=horizon,
#            target_col_idx=segment_combined.shape[1]-1,
#            shuffle=False
#        )
#        Xw_all = Xw_all[:, :, :n_feat]
#
#        seg_len = len(segment_combined)
#        first_target_local = window_size + horizon - 1
#        local_target_indices = np.arange(first_target_local, seg_len)
#        global_target_indices = start + local_target_indices
#
#        mask_test = global_target_indices >= end_train
#        mask_train = ~mask_test
#
#        Xw_tr, yw_tr = Xw_all[mask_train], yw_all[mask_train]
#        Xw_te, yw_te = Xw_all[mask_test], yw_all[mask_test]
#
#        scaler = StandardScaler()
#        Xw_tr_2d = Xw_tr.reshape(-1, n_feat)
#        Xw_te_2d = Xw_te.reshape(-1, n_feat)
#        scaler.fit(Xw_tr_2d)
#        Xw_tr = scaler.transform(Xw_tr_2d).reshape(Xw_tr.shape[0], window_size, n_feat)
#        Xw_te = scaler.transform(Xw_te_2d).reshape(Xw_te.shape[0], window_size, n_feat)
#
#        class_weight = make_class_weights(yw_tr) if use_class_weights else None
#
#        model = build_model_fn((window_size, n_feat))
#        model.fit(
#            Xw_tr, yw_tr,
#            epochs=epochs, batch_size=batch_size,
#            verbose=verbose,
#            class_weight=class_weight,
#            callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)]
#        )
#
#        proba = model.predict(Xw_te, verbose=0).ravel()
#        preds = (proba >= threshold).astype(int)
#
#        y_true_all.extend(yw_te.tolist())
#        y_proba_all.extend(proba.tolist())
#        y_pred_all.extend(preds.tolist())
#        fold_id_all.extend([fold]*len(yw_te))
#
#        folds_info.append({
#            "fold": fold,
#            "train_start": start,
#            "train_end": end_train-1,
#            "test_start": end_train,
#            "test_end": end_test-1,
#            "n_train_windows": len(Xw_tr),
#            "n_test_windows": len(Xw_te),
#        })
#
#        start += step_size
#
#    y_true_all = np.array(y_true_all)
#    y_pred_all = np.array(y_pred_all)
#    y_proba_all = np.array(y_proba_all)
#
#    metrics = {
#        "accuracy": accuracy_score(y_true_all, y_pred_all),
#        "f1": f1_score(y_true_all, y_pred_all),
#        "precision": precision_score(y_true_all, y_pred_all),
#        "recall": recall_score(y_true_all, y_pred_all),
#        "roc_auc": roc_auc_score(y_true_all, y_proba_all) if len(np.unique(y_true_all)) == 2 else np.nan
#    }
#
#    preds_df = pd.DataFrame({
#        "y_true": y_true_all,
#        "y_proba": y_proba_all,
#        "y_pred": y_pred_all,
#        "fold": fold_id_all
#    })
#
#    folds_df = pd.DataFrame(folds_info)
#    return metrics, preds_df, folds_df
#
#def walk_forward_with_threshold_tuning(
#    X_values, y_values,
#    window_size=30, horizon=1,
#    train_size=720, test_size=30, step_size=30,
#    build_model_fn=build_lstm,
#    epochs=50, batch_size=64,
#    val_frac_in_train=0.15,   # fracción del train usada como validation (últos días)
#    lr=1e-3,
#    use_class_weights=True,
#    verbose=0
#):
#    n = len(X_values)
#    n_feat = X_values.shape[1]
#
#    y_true_all, y_pred_all, y_proba_all = [], [], []
#
#    start = 0
#    fold = 0
#    while start + train_size + test_size <= n:
#        fold += 1
#        end_train = start + train_size
#        end_test = end_train + test_size
#
#        segment_X = X_values[start:end_test]
#        segment_y = y_values[start:end_test]
#
#        segment_combined = np.hstack([segment_X, segment_y.reshape(-1,1)])
#        Xw_all, yw_all = create_windows_multivariate_np(
#            data=segment_combined, window_size=window_size, horizon=horizon,
#            target_col_idx=segment_combined.shape[1]-1, shuffle=False
#        )
#        Xw_all = Xw_all[:, :, :n_feat]
#
#        seg_len = len(segment_combined)
#        first_target_local = window_size + horizon - 1
#        local_target_indices = np.arange(first_target_local, seg_len)
#        global_target_indices = start + local_target_indices
#
#        mask_test = global_target_indices >= end_train
#        mask_train = ~mask_test
#
#        Xw_tr, yw_tr = Xw_all[mask_train], yw_all[mask_train]
#        Xw_te, yw_te = Xw_all[mask_test], yw_all[mask_test]
#
#        # split validation from the END of train (temporal)
#        n_tr = len(Xw_tr)
#        n_val = int(np.ceil(val_frac_in_train * n_tr))
#        n_train_effective = n_tr - n_val
#        X_tr_eff, y_tr_eff = Xw_tr[:n_train_effective], yw_tr[:n_train_effective]
#        X_val, y_val = Xw_tr[n_train_effective:], yw_tr[n_train_effective:]
#
#        # scaler fit only on X_tr_eff
#        scaler = StandardScaler()
#        X_tr_eff_2d = X_tr_eff.reshape(-1, n_feat)
#        scaler.fit(X_tr_eff_2d)
#        X_tr_eff = scaler.transform(X_tr_eff_2d).reshape(X_tr_eff.shape[0], window_size, n_feat)
#
#        X_val = scaler.transform(X_val.reshape(-1, n_feat)).reshape(X_val.shape[0], window_size, n_feat)
#        Xw_te_scaled = scaler.transform(Xw_te.reshape(-1, n_feat)).reshape(Xw_te.shape[0], window_size, n_feat)
#
#        class_weight = make_class_weights(y_tr_eff) if use_class_weights else None
#
#        # Build model and set optimizer lr
#        model = build_model_fn((window_size, n_feat))
#        # override optimizer lr if using Adam
#        try:
#            tf.keras.backend.set_value(model.optimizer.lr, lr)
#        except Exception:
#            pass
#
#        callbacks = [
#            EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=0),
#            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=0)
#        ]
#
#        model.fit(
#            X_tr_eff, y_tr_eff,
#            validation_data=(X_val, y_val),
#            epochs=epochs, batch_size=batch_size,
#            class_weight=class_weight,
#            callbacks=callbacks, verbose=verbose
#        )
#
#        # Optimizamos threshold en validation (buscar threshold que maximiza F1)
#        val_proba = model.predict(X_val, verbose=0).ravel()
#        best_th, best_f1 = 0.5, f1_score(y_val, (val_proba>=0.5).astype(int))
#        for th in np.linspace(0.3, 0.7, 41):  # explora 0.30..0.70 cada 0.01
#            f1v = f1_score(y_val, (val_proba>=th).astype(int))
#            if f1v > best_f1:
#                best_f1 = f1v
#                best_th = th
#
#        # predecir test con ese threshold
#        proba_test = model.predict(Xw_te_scaled, verbose=0).ravel()
#        preds_test = (proba_test >= best_th).astype(int)
#
#        y_true_all.extend(yw_te.tolist())
#        y_proba_all.extend(proba_test.tolist())
#        y_pred_all.extend(preds_test.tolist())
#
#        start += step_size
#
#    # metrics globales
#    from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
#    y_true_all = np.array(y_true_all)
#    y_pred_all = np.array(y_pred_all)
#    y_proba_all = np.array(y_proba_all)
#
#    metrics = {
#        "accuracy": accuracy_score(y_true_all, y_pred_all),
#        "f1": f1_score(y_true_all, y_pred_all),
#        "precision": precision_score(y_true_all, y_pred_all),
#        "recall": recall_score(y_true_all, y_pred_all),
#        "roc_auc": roc_auc_score(y_true_all, y_proba_all)
#    }
#    return metrics
#
## -----------------------------
## === EJECUCIÓN SOBRE df_bitcoin ===
## -----------------------------
## X = todas las columnas menos target
#df_bitcoin["Open time"] = pd.to_datetime(df_bitcoin["Open time"])
#df_bitcoin = df_bitcoin.set_index("Open time")
#
#X_values = df_bitcoin.drop(columns=["Target"]).values
#y_values = df_bitcoin["Target"].values
#
#
#lookbacks = [14, 30, 60]
#horizon = 1
#train_size = 720
#test_size = 30
#step_size = 30
#
#results = []
#
#for lb in lookbacks:
#    for model_name, builder in MODEL_BUILDERS_IMPROVED.items():
#        print(f"\n>>> {model_name} | lookback={lb}")
#        metrics = walk_forward_with_threshold_tuning(
#            X_values=X_values,
#            y_values=y_values,
#            window_size=lb,
#            horizon=horizon,
#            train_size=train_size,
#            test_size=test_size,
#            step_size=step_size,
#            build_model_fn=builder,
#            epochs=50,
#            batch_size=64,
#            val_frac_in_train=0.15,
#            lr=1e-3,
#            use_class_weights=True,
#            verbose=1
#        )
#        results.append({"model": model_name, "lookback": lb, **metrics})
#
#results_df = pd.DataFrame(results).sort_values(["roc_auc","f1","accuracy"], ascending=False)
#print("\n=== RESULTADOS ===")
#print(results_df)